<a href="https://colab.research.google.com/github/sfbllgrn/DD2412_Class_Contrastive_Explanations/blob/main/experiment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/sfbllgrn/DD2412_Class_Contrastive_Explanations.git

import os
import shutil

# Define the source and destination paths
source_folder = '/content/DD2412_Class_Contrastive_Explanations'
destination_folder = '/content'

# List the files and subdirectories in the source folder
contents = os.listdir(source_folder)

# Move each item from the source folder to the destination folder
for item in contents:
    source_path = os.path.join(source_folder, item)
    destination_path = os.path.join(destination_folder, item)
    shutil.move(source_path, destination_path)

# Remove the now-empty source folder
os.rmdir(source_folder)


In [11]:
# Mount Google drive that contains all data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [125]:
# Imports
from torchvision.models import densenet161, DenseNet161_Weights
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
from torchvision.models import alexnet, AlexNet_Weights
from torchvision.models import googlenet, GoogLeNet_Weights
from torchvision.models import mnasnet0_5, MNASNet0_5_Weights # Här gissar jag att dom använder 0.5, står inte någonstans
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
from torchvision.models import efficientnet_b1, EfficientNet_B1_Weights

import numpy as np
import torch
from torch.autograd.functional import jacobian as J
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import sys
from torchvision import transforms
from torch import nn
from torch.nn.functional import one_hot


In [115]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [45]:
# Load data

# when data is local
#data_folder = "/Users/sofia/Documents/Skola/KTH/Master/Deep Learning, Advanced Course DD2412/Class Contrastive Explanations/DD2412_Class_Contrastive_Explanations/Data_small"

# data on google drive
data_folder = "/content/drive/MyDrive/Colab Notebooks/Deep learning advanced/ImageNet_Data/val"
data_obj = ImageFolder(root=data_folder, transform=DenseNet161_Weights.DEFAULT.transforms())

BATCH_SIZE = 1
val_dataloader = DataLoader(data_obj, batch_size=BATCH_SIZE, shuffle=False)


In [42]:
# Init Pretrained models

densenet = densenet161(weights=DenseNet161_Weights.IMAGENET1K_V1)
mobilenet_small = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
alex = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
google = googlenet(weights=GoogLeNet_Weights.IMAGENET1K_V1)
mnasnet = mnasnet0_5(weights=MNASNet0_5_Weights.IMAGENET1K_V1)
resnet = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
mobilenet_large = mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1)  # Denna har även IMAGENET1K_v2
efficientnet = efficientnet_b1(weights=EfficientNet_B1_Weights.IMAGENET1K_V1) # Denna har även IMAGENET1K_v2
pretrained_models = {
                     "alexnet":alex, "googlenet":google,
                     "mnasnet":mnasnet, "resnet":resnet,
                     "mobilenet_large":mobilenet_large,
                     "efficientnet":efficientnet,
                     "densenet":densenet, "mobilenet_small":mobilenet_small}

In [126]:
# Perform gradient sign pertubations

def backward_gradient_explanation(X, label, net, probs=False):
  logits = net(X)
  pred_probab = nn.Softmax(dim=1)(logits)
  yt_oh = one_hot(label, num_classes=logits.shape[1])
  external_grad = torch.reshape(yt_oh, logits.shape)
  X.retain_grad()
  if probs:
    pred_probab.backward(gradient=external_grad)
    return X.grad

  logits.backward(gradient=external_grad)

  return X.grad

def attribution_explanation(net, x, pred_indx):
  value_logits = J(lambda x:net(x)[np.arange(BATCH_SIZE), pred_indx],x)
  value_logits = torch.diagonal(value_logits)
  value_logits = value_logits.permute(3,0,1,2)
  #value_probs = J(lambda x:torch.nn.functional.softmax(net(x), dim=1)[np.arange(BATCH_SIZE),pred_indx], x)
  #value_probs = torch.diagonal(value_probs)
  #value_probs = value_probs.permute(3,0,1,2)
  return value_logits


def gradient_sign_pertube(inputs, labels, net, n):
  epsilon = 1e-3
  x_logits = inputs.clone()
  alpha = epsilon/n
  for i in range(n):
    # Att tänka ut: ska det vara x eller data nedan
    logits = backward_gradient_explanation(inputs, labels, net, probs=False)
    x_logits = x_logits + alpha*torch.sign(logits)  # blir det rätt index här?
    x_logits = torch.clamp(x_logits, min=torch.minimum(inputs-epsilon, torch.tensor(0)), max=torch.maximum(inputs+epsilon, torch.tensor(1)))

  return x_logits


## Test


In [130]:
torch.manual_seed(1)

n=1

#for name, model in pretrained_models.items():

name = "alex"
model = alex.to(device)

# Set model to eval mode
model.eval()

subset_size = len(data_obj)
subset_size = 100

correct = {"probs":0, "logits":0, "unperturbed":0}
total = {"probs":0, "logits":0, "unperturbed":0}
changes = {"yt":[], "pt":[]}

#with torch.no_grad():
correct_pert = {"probs":0, "logits":0, "unperturbed":0}
total_pert = 0
for batch_idx, (inputs, labels) in enumerate(val_dataloader):
    inputs.requires_grad_(True)
    inputs = inputs.to(device)
    labels = labels.to(device)
    perturbed_x_logits = gradient_sign_pertube(inputs, labels, model, n)

    if batch_idx%subset_size/10 == 0:
      print(batch_idx/subset_size)

    if batch_idx < subset_size:
        y_pert = model(perturbed_x_logits)
        y = model(inputs)

        yt_pert = y_pert[np.arange(BATCH_SIZE), labels]
        yt = y[np.arange(BATCH_SIZE), labels]

        changes['yt'].append(yt_pert-yt)

        pt_pert = torch.nn.functional.softmax(y_pert, dim=1)[np.arange(BATCH_SIZE), labels]
        pt = torch.nn.functional.softmax(y, dim=1)[np.arange(BATCH_SIZE), labels]
        changes['pt'].append(pt_pert-pt)


        #_, predicted_probs = torch.max(outputs_probs, 1)
        _, predictions_pert = torch.max(y_pert, 1)
        _, predictions_unperturbed = torch.max(y, 1)
        total_pert += labels.size(0)
        correct_pert["logits"] += (predictions_pert == labels).sum().item()
        #correct_pert["probs"] += (predicted_probs == labels).sum().item()
        correct_pert["unperturbed"] += (predictions_unperturbed == labels).sum().item()

    else:
        break

accuracy_logits = correct_pert['logits']/ total_pert
#accuracy_probs = correct_pert['probs']/ total_pert
accuracy_unperturbed = correct_pert['unperturbed']/ total_pert

print("Average changes in yt: {} and pt: {}".format(torch.mean(torch.stack(changes['yt'])), torch.mean(torch.stack(changes['pt']))))
print('Validation Accuracy for {} with pertubed data on logits, {} iterations: {}\%'.format(name, n, accuracy_logits*100))
#print('Validation Accuracy for {} with pertubed data on probs, {} iterations: {}\%'.format(name, n, accuracy_probs*100))
print('Validation Accuracy for {} with pertubed data on unperturbed data, {} iterations: {}\%'.format(name, n, accuracy_unperturbed*100))

0.0


OutOfMemoryError: ignored